## Predicting late-onset sepsis  in preterm infants using temperature data

By Paula Romero Jiménez

Outline of the project:

- Preprocess and merge the 3 datasets used: patient characteristics, temperature data and blood culture data.
- Feature engineering for the model: differences and variation in temperature over time.
- LOS labeling.
- Calculate baseline characteristics.
- Matching of patients.
- Training of the model: Logistic Regression (binary classification of LOS or no LOS).
- Model evaluation.

First import the required packages:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.calibration import calibration_curve
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

### 1. Preprocess and merge the 3 datasets

The main and most important one is the one with the temperature data because it will be the main  feature in  the model, I won't use patients who don't have temeprature data

In [ ]:
data_raw = pd.read_csv("ads2024/Signals_5488_ads2024.csv")

We filter out temepratures below 35 degrees (hypothermic) and above 39.5 degrees (hyperthermic). We also filter out negative times and times surpassing the 30 days.

In [ ]:
data =  data_raw[(data_raw['Value'] > 35) & (data_raw['Value'] < 39.5)]
data = data[(data['Time'] > 0) & (data['Time'] <= (30*24*60))]
del data['Unnamed: 0']
del data['ParameterID']
del data['parameterName']
del data['UnitName']
del data['UnitMultiplier']
print('Number of patients:', data['pt_id'].nunique())

In [ ]:
data

We filter out patients with  age at admission greater than  48 hours, the first 'Time' measurement is considered to be the age of admission (because it's minutes since birth)

In [ ]:
data['Time'] = pd.to_datetime(data['Time'], unit = 'm')
data.sort_values(by = ['pt_id','Time'], inplace= True)
age_admission = data.groupby('pt_id')['Time'].first().reset_index()
age_admission.columns = ['pt_id','age_at_admission']
age_admission['age_at_admission'].dt.day.unique()

elegible_patients = age_admission[age_admission['age_at_admission'].dt.day <= 2]['pt_id'].unique()
data = data[data['pt_id'].isin(elegible_patients)]
print('Number of patients:', data['pt_id'].nunique())

We won't use the temperature data in minutes, if not by hours, so we perform the aggregation first.

In [ ]:
data.set_index('Time', inplace=True)

agg_data = data.groupby('pt_id').resample('1H').agg({
    'Value':['mean', 'std'],
}).reset_index()

agg_data.columns = ['pt_id', 'Time', 'temp_mean', 'temp_std']

In [ ]:
# There are hours where we don't have temeprature values, so we delete those missing values that we have created with the aggregation function
agg_data = agg_data.dropna(subset = ['temp_mean'])
agg_data

We will now merge this temperature data with the blood culture data. We use the patient ID and the time to merge it.

In the blood culture dataset we filter out negative times: there are quite a lot of them, which indicates that they were incorrectly annotated. Nonetheless, by looking at the number of patients with positive blood culture, we don't lose too much information.

In [ ]:
bc = pd.read_csv("ads2024/bloodcultures_ads2024.csv")

In [ ]:
print('Initial number of patients with blood culture:', bc['pt_id'].nunique())
del bc['Unnamed: 0']
del bc['filter_specimen_datetime']
bc1 =  bc[(bc['Time'] < 0)&(bc['bloodculture_result']==1)]['pt_id'].nunique()
bc = bc[(bc['Time'] > 0)]
print('Number of patients with blood culture after filtering negative time values:', bc['pt_id'].nunique())
print('Patients with negative time values and positive blood culture:', bc1)

In [ ]:
bc

In [ ]:
bc['Time'] = pd.to_datetime(bc['Time'],unit='m').dt.floor('H')

The join operation will be left because if for that patient we don't have a blood culture, we assume it was because that patient didn't have sepsis syntoms and the clinician didn't ask for the test. We want to still mantain the temperature data of these patients which will be treated as controls.

In [ ]:
data2 = pd.merge(agg_data, bc, on = ['pt_id','Time'], how = 'left')
data2

In [ ]:
print('Number of patients present in temperature dataset but not in the blood culture dataset:',len(set(agg_data['pt_id'].unique()) - set(bc['pt_id'].unique())))

In [ ]:
npat = data2[data2['bloodculture_result'].notnull()]['pt_id'].nunique()
print('Patients with blood culture result in the merged dataframe:', npat)

We lose patients with blood culture result in the merging due to different patient IDs and different times.

Now we'll merge with patient characteristics to obtain gestational age and gender.


In [ ]:
patient = pd.read_csv("ads2024/patient_characteristics_ads2024.csv")

In [ ]:
patient['GestationalAgeDays'] = patient['GestationalAgeDays'].fillna(0)
del patient['Unnamed: 0']
del patient['birth_datetime']
del patient['Death']
del patient['DeathDate']
patient['gestational_age'] = patient['GestationalAgeWeeks']*7 + patient['GestationalAgeDays']
print('Number of patients:', patient['pt_id'].nunique())

In [ ]:
patient

In [ ]:
data3 = pd.merge(data2, patient, on = 'pt_id', how = 'inner')
data3

In [ ]:
data3['pt_id'].nunique()

The issue now is that we have more than one positive blood culture result per patient and we want to keep the first one, so  we will filter the rest.

In [ ]:
data3['is_positive'] = data3['bloodculture_result'] == 1
first_positive_pp = data3.groupby('pt_id')['is_positive'].apply(lambda x: x.cumsum() == 1)
final_data = data3[~data3['is_positive']|first_positive_pp]
final_data = final_data.drop('is_positive', axis=1)

In [ ]:
set1 = set(final_data[(final_data['minutes_since_birth'] >= 4320) & (final_data['bloodculture_result'] == 1)]['pt_id'].unique())
print('LOS patients (positive blood culture after 72h):',len(set1))

In [ ]:
set2 = set(final_data[(final_data['minutes_since_birth'] < 4320) & (final_data['bloodculture_result'] == 1)]['pt_id'].unique())
print('Patients with positive blood culture before 72h:',len(set2))

In [ ]:
final_data

### 2. Feature engineering

We calculate the rolling mean and standard deviation for a window of 12 hours. For the initial 12 hours we use the mean of the previous available hours.

In [ ]:
def rolling_stats(df):
    #expanding mean and std for initial 12 hours
    df['expanding_mean'] = df['temp_mean'].expanding(min_periods = 1).mean().shift(1)
    df['expanding_std'] = df['temp_std'].expanding(min_periods = 1).mean().shift(1)
    
    #rolling mean and std for subsequent hours
    df['rolling_mean'] = df['temp_mean'].rolling(window = 12, min_periods = 1).mean().shift(1)
    df['rolling_std'] = df['temp_std'].rolling(window = 12, min_periods = 1).mean().shift(1)
    
    #combine both calculations
    df['rolling_mean'] = df['rolling_mean'].combine_first(df['expanding_mean'])
    df['rolling_std'] = df['rolling_std'].combine_first(df['expanding_std'])
    
    #drop intermediate columns
    df.drop(columns = ['expanding_mean','expanding_std'], inplace = True)
    
    return df

In [ ]:
final_data = final_data.groupby('pt_id').apply(rolling_stats).reset_index(drop = True)

Function to calculate temperature differences with the computed means over time. 

In [ ]:
def temp_differences(df):
    df['temp_mean_diff'] = (df['temp_mean'] - df['rolling_mean']).abs()
    df['temp_std_diff'] = (df['temp_std'] - df['rolling_std']).abs()
    return df

In [ ]:
final_data = final_data.groupby('pt_id').apply(temp_differences).reset_index(drop = True)

Fill missing values with 0: first values for each patients are going to be missing.

In [ ]:
final_data['temp_mean_diff'] = final_data['temp_mean_diff'].fillna(0)
final_data['temp_std_diff'] = final_data['temp_std_diff'].fillna(0)
final_data['bloodculture_result'] = final_data['bloodculture_result'].fillna(0)

In [ ]:
final_data

### 3. LOS Labeling

Los patients: patients with positive blood culture after 72h of birth.

Control patients: patients with positive blood culture before 72h, patients with negative blood cultures and patients without blood cultures.

Also for modeling purposes, we will take temperature measures before 6h of sepsis onset (t=0, blood culture drawn) and 2h after, and label them as LOS

In [ ]:
first_positive = final_data[final_data['bloodculture_result'] == 1].sort_values(by = 'Time').groupby('pt_id')['Time'].first().reset_index()
first_positive.columns = ['pt_id', 'first_positive_time']
final_data = pd.merge(final_data, first_positive, on = ['pt_id'], how='left')
final_data['first_positive_time'] = pd.to_datetime(final_data['first_positive_time'], unit = 'm')

In [ ]:
def label_los(df):
    control_mask = (df['first_positive_time'].isna()) | (df['first_positive_time'].dt.day < 3)
    label_mask = ((df['first_positive_time'].dt.day > 3) & 
                  (df['Time'] >= (df['first_positive_time'] - pd.Timedelta(hours=6))) & 
                  (df['Time'] <= (df['first_positive_time'] + pd.Timedelta(hours=2))))

    df['los_label'] = 0
    df.loc[control_mask, 'los_label'] = 0
    df.loc[label_mask, 'los_label'] = 1

    return df['los_label']
    
    
final_data['los_label'] = label_los(final_data)

In [ ]:
final_data

In [ ]:
patient_counts = final_data.groupby('pt_id')['los_label'].max().value_counts()

plt.bar(['No LOS', 'LOS'], patient_counts, color = ['blue', 'orange'])
plt.ylabel('Number of patients')
plt.title('Distribution of patients with and without LOS')
plt.show()
print(patient_counts)

In [ ]:
grouped = final_data.groupby(['pt_id','los_label']).first().reset_index()
mean = grouped.groupby('los_label')['gestational_age'].mean()
mean.columns = ['los_label','mean']
mean

### 4. Calculate baseline characteristics

In [ ]:
hosp_duration = final_data.groupby('pt_id')['Time'].apply(lambda x: x.max()-x.min())
los_patients = final_data[final_data['los_label'] == 1]['pt_id'].unique()
los_data = final_data[final_data['pt_id'].isin(los_patients)]
control_data = final_data[~final_data['pt_id'].isin(los_data['pt_id'])]

In [ ]:
los_id = (los_data['pt_id'].unique()).tolist()
los_ga = [group.iloc[0] for pt_id, group in los_data.groupby('pt_id')['gestational_age']]
los_temp_mean = [group.mean() for pt_id, group in los_data.groupby('pt_id')['temp_mean']]
los_temp_std = [group.mean() for pt_id, group in los_data.groupby('pt_id')['temp_std']]
los_temp_mean_diff = [group.mean() for pt_id, group in los_data.groupby('pt_id')['temp_mean_diff']]
los_temp_std_diff = [group.mean() for pt_id, group in los_data.groupby('pt_id')['temp_std_diff']]
los_hosp = list(los_data.groupby('pt_id')['Time'].apply(lambda x: x.max()-x.min()))

los_df = pd.DataFrame({
    'pt_id': los_id,
    'gestational_age': los_ga,
    'temp_mean': los_temp_mean,
    'temp_std': los_temp_std,
    'temp_mean_diff': los_temp_mean_diff,
    'temp_std_diff': los_temp_std_diff,
    'hospitalization': los_hosp,
})

In [ ]:
los_df

In [ ]:
non_los_id = (control_data['pt_id'].unique()).tolist()
non_los_ga = [group.iloc[0] for pt_id, group in control_data.groupby('pt_id')['gestational_age']]
non_los_temp_mean = [group.mean() for pt_id, group in control_data.groupby('pt_id')['temp_mean']]
non_los_temp_std = [group.mean() for pt_id, group in control_data.groupby('pt_id')['temp_std']]
non_los_temp_mean_diff = [group.mean() for pt_id, group in control_data.groupby('pt_id')['temp_mean_diff']]
non_los_temp_std_diff = [group.mean() for pt_id, group in control_data.groupby('pt_id')['temp_std_diff']]
non_los_hosp = list(control_data.groupby('pt_id')['Time'].apply(lambda x: x.max()-x.min()))

control_df = pd.DataFrame({
    'pt_id': non_los_id,
    'gestational_age': non_los_ga,
    'temp_mean': non_los_temp_mean,
    'temp_std': non_los_temp_std,
    'temp_mean_diff': non_los_temp_mean_diff,
    'temp_std_diff': non_los_temp_std_diff,
    'hospitalization': non_los_hosp,
})

In [ ]:
control_df

In [ ]:
from scipy.stats import shapiro, ttest_ind, mannwhitneyu, chi2_contingency

cont_vars = ['gestational_age', 'temp_mean', 'temp_std', 'temp_mean_diff', 'temp_std_diff', 'hospitalization']
categ_vars = ['Gender']

def test_normality(data):
    stat,p = shapiro(data)
    return p > 0.05 # True if normal distribution

for var in cont_vars:
    is_normal_los = test_normality(los_df[var])
    is_normal_control = test_normality(control_df[var])
    print(f'{var}: LOS group normality = {is_normal_los}, control group normality = {is_normal_control}')

In [ ]:
for var in cont_vars:
    is_normal = test_normality(los_df[var])
    if is_normal:
        mean = los_df[var].mean()
        std = los_df[var].std()
        print(var, mean, std)
    else:
        median = los_df[var].median()
        q1 = los_df[var].quantile(0.25)
        q3 = los_df[var].quantile(0.75)
        print(var, median, q1,q3)

In [ ]:
for var in cont_vars:
    is_normal = test_normality(control_df[var])
    if is_normal:
        mean = control_df[var].mean()
        std = control_df[var].std()
        print(var, mean, std)
    else:
        median = control_df[var].median()
        q1 = control_df[var].quantile(0.25)
        q3 = control_df[var].quantile(0.75)
        print(var, median, q1,q3)

In [ ]:
for var in cont_vars:
    los_data_var = los_df[var]
    control_data_var = control_df[var]
    los_data_var = los_df[var]
    is_normal_los = test_normality(los_data_var)
    is_normal_control = test_normality(control_data_var)
    
    if is_normal_los and is_normal_control:
        t, p = ttest_ind(los_data_var, control_data_var)
        print(var, 'Students t-test, p-value:', p)
    else:
        u,p = mannwhitneyu(los_data_var, control_data_var)
        print(var, 'Mann-Whitney U test:', p)

In [ ]:
gender_los = list(los_data.groupby('pt_id')['Gender'].unique().value_counts())
gender_control = list(control_data.groupby('pt_id')['Gender'].unique().value_counts())

In [ ]:
contingency = pd.DataFrame({
    'LOS': gender_los,
    'CONTROL': gender_control
})
chi2, p, dof, expected = chi2_contingency(contingency)
print('Chi-square test for gender', p)

In [ ]:
los_df['LOS_label'] = 1
control_df['LOS_label'] = 0
data_pp = pd.concat([los_df, control_df], ignore_index=True)

In [ ]:
data_pp

### 5. Matching of patients

We will match LOS patients to controls based on gestational age, gender and time.

In [ ]:
final_data

In [ ]:
los = final_data[(final_data['los_label'] == 1)&(final_data['bloodculture_result'] == 1)].groupby('pt_id').first().reset_index()
non_los = final_data[~final_data['pt_id'].isin(los['pt_id'])]
los = los[['pt_id','Time','bloodculture_result','gestational_age','Gender','los_label','first_positive_time']]
non_los = non_los[['pt_id','Time','bloodculture_result','gestational_age','Gender','los_label']]

In [ ]:
los

In [ ]:
matched = pd.merge(los, non_los, on = ['Time','Gender'], suffixes = ('_los', '_control'))
matched = matched[
    (matched['gestational_age_control'] >= (matched['gestational_age_los']-2)) &
    (matched['gestational_age_control'] <= (matched['gestational_age_los']+2))    
]

In [ ]:
matched

In [ ]:
used_controls = set()
final_pairs = pd.DataFrame()

for pt_id_los in los['pt_id'].unique():
    
    group = matched[matched['pt_id_los'] == pt_id_los]
    
    selected_matches = group[~group['pt_id_control'].isin(used_controls)].head(4)
    
    final_pairs = pd.concat([final_pairs, selected_matches], ignore_index = True)
    
    used_controls.update(selected_matches['pt_id_control'])

print(final_pairs['pt_id_los'].nunique(),'LOS patients matched to', final_pairs['pt_id_control'].nunique(),'control patients')

An interesting thing would be to select the time for control patients, to address the class imbalance at the time of modelling. Therefore, we extract the time LOS patients are labelled as LOS (-6 to +2) and match it for the already matched patients.

In [ ]:
def extract_time(df, matched_pairs):
    relevant_rows = pd.DataFrame()
    
    for _,row in matched_pairs.iterrows():
        los_time = df[(df['pt_id'] == row['pt_id_los'])&(df['los_label'] == 1)]['Time']
        
        control_time = df[(df['pt_id'] == row['pt_id_control'])&(df['Time'].isin(los_time))]
        control_time['matched_pair'] = f"{row['pt_id_los']}"
        
        relevant_rows = pd.concat([relevant_rows, control_time], ignore_index=True)
    return relevant_rows

In [ ]:
control_times = extract_time(final_data, final_pairs)
los_times = final_data[final_data['pt_id'].isin(final_pairs['pt_id_los']) & (final_data['los_label']==1)]
final_df = pd.concat([los_times, control_times])

In [ ]:
final_df

In [ ]:
final_df['pt_id'].nunique()

### 6. Logistic regression

We train the model with the matched patients.

In [ ]:
final_df['Gender'] = final_df['Gender'].map({'M':0,'F':1}) 
features = ['temp_mean_diff','temp_std_diff','gestational_age', 'Gender']
target = 'los_label'

X = final_df[features]
y = final_df[target]

train_idx, test_idx = next(StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42).split(X, y, groups = final_df['pt_id']))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx] 
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


inner_cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
outer_cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

param_grid ={'C':[0.01, 0.1, 1, 10, 100],'penalty':['l2'], 'solver':['newton-cg', 'lbfgs','liblinear'],'class_weight':['balanced', None]}
grid_search = GridSearchCV(LogisticRegression(max_iter = 1000), param_grid, scoring='roc_auc', cv = inner_cv, n_jobs=-1)
groups = final_df.iloc[train_idx]['pt_id']
grid_search.fit(X_train, y_train, groups = groups)
best_params = grid_search.best_params_


In [ ]:
best_params

In [ ]:
model = LogisticRegression(**best_params, max_iter=1000)
model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
auc_score = roc_auc_score(y_test, y_pred_prob)
fraction_pos, mean_predicted_value = calibration_curve(y_test,y_pred_prob, n_bins=10)
precision = precision_score(y_test, y_pred_prob>0.5, zero_division = 'warn')
recall = recall_score(y_test, y_pred_prob>0.5)

In [ ]:
precision

In [ ]:
recall

In [ ]:
model.coef_[0]

In [ ]:
X_train_sm = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_sm).fit()
p_values = logit_model.pvalues[1:]

In [ ]:
for feature, pval in zip(features,p_values):
    print(f'{feature}: p-value: {pval}')

In [ ]:
logit_model.params

In [ ]:
mean_predicted_value

In [ ]:
print(auc_score)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(mean_predicted_value, fraction_pos, marker='o', label='Calibration curve' )
plt.plot([0,1], [0,1], linestyle='--', color='gray', label='Perfectly calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of positives')
plt.title('Calibration Plot')
plt.legend()
plt.savefig("calibration_plot2.png")
plt.show()

In [ ]:
from sklearn.metrics import brier_score_loss

brier_score_loss(y_test, y_pred_prob)

Visualize class distribution

In [ ]:
class_distribution = final_df['los_label'].value_counts()

plt.figure(figsize = (6,4))
class_distribution.plot(kind = 'bar', color = ['blue', 'orange'] )
plt.title('Class distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.xticks(range(len(class_distribution)), ['Negative (0)', 'Positive (1)'], rotation = 0 )
plt.savefig('class_distribution.png')
plt.show()

print(class_distribution)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='blue', lw = 2, label=f'AUC ={auc_score:.3f}' )
plt.plot([0,1], [0,1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend()
plt.grid()
plt.savefig("roc_curve.png")
plt.show()